In [ ]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta
import datetime as datetime
import sqlite3
import base64
from requests import post, get
import pytz
import psycopg2
import sqlalchemy
from sqlalchemy.orm import sessionmaker

In [ ]:
time_now = datetime.datetime.now()
print(time_now)

In [ ]:
# Your Spotify API credentials
def get_token():
    # Spotify API endpoints
    AUTH_URL = 'https://accounts.spotify.com/authorize'
    TOKEN_URL = 'https://accounts.spotify.com/api/token'
    API_URL = 'https://api.spotify.com/v1/me/player/recently-played'

    params = {
        'limit':50,
    }

    # Your Spotify API credentials
    CLIENT_ID = ''
    CLIENT_SECRET = ''
    REDIRECT_URI = ''
    SCOPE = ''

    # Step 1: Generate the authorization URL
    auth_params = {
        'client_id': CLIENT_ID,
        'response_type': 'code',
        'redirect_uri': REDIRECT_URI,
        'scope': SCOPE,
    }

    auth_url = f"{AUTH_URL}?client_id={auth_params['client_id']}&response_type={auth_params['response_type']}&redirect_uri={auth_params['redirect_uri']}&scope={auth_params['scope']}"
    print("Please visit the following URL to authorize your application:")
    print(auth_url)

    # After the user authorizes your application, you'll receive the authorization code via the REDIRECT_URI.
    authorization_code = input("Enter the authorization code from the URL: ")

    # Step 2: Exchange Authorization Code for Access Token
    token_data = {
        'grant_type': 'authorization_code',
        'code': authorization_code,
        'redirect_uri': REDIRECT_URI,
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
    }

    token_response = requests.post(TOKEN_URL, data=token_data)

    if token_response.status_code == 200:
        access_token = token_response.json().get('access_token')
        return access_token

token = get_token()
if token:
    print("Token Retrieved Successfully!!")
else:
    print("Failed to fetch token!!")

In [ ]:
def check_valid_data(dataframe: pd.DataFrame) -> bool:
    if dataframe.empty:
        print("No songs listened to within the last 24 hours")
        return False
#     primary key check
    if pd.Series(dataframe['Played_At']).is_unique:
        pass
    else:
        raise Exception("Primary Key Check is violated")
#         check for nulls
    if dataframe.isnull().values.any():
        raise Exception("Null Values Found!")
#         ensure the data is only from 24 hours ago
    today = datetime.datetime.now()
    hours24_ago = timedelta(days=1)
    diff = today - hours24_ago
    timestamps = dataframe["Played_At"].tolist()

    for timestamp in song_dict["Played_At"]:
         if datetime.datetime.strptime(timestamp, "%H:%M:%S") > diff:
            raise Exception("Data is not entirely from the last 24 hours, timestamp")
   
    return True

In [ ]:
if __name__ == "__main__":

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": "Bearer {token}".format(token=token)
    }
    params = {
        'limit':50,
    }

    today = datetime.datetime.now()
    hours24_ago = timedelta(days=1)
    time_24 = today - hours24_ago
    hours24_ago_timestamp = int(time_24.timestamp()) * 1000
    all_data = []

    result = requests.get("https://api.spotify.com/v1/me/player/recently-played?after={time}"
                       .format(time=hours24_ago_timestamp), params = params, headers=headers)
    json_result = result.json()

            
    songs =[]
    artists_name =  []
    played_at = []
    timestamps = []

    for song in json_result["items"]:
        songs.append(song["track"]["name"])
        artists_name.append(song["track"]["album"]["artists"][0]["name"])
        played_at.append(song["played_at"])
        timestamps.append(song["played_at"][0:10])

    song_dict = {
        "Track": songs,
        "Artist": artists_name,
        "Played_At": played_at,
        "Date": timestamps
    }
    source_tz = pytz.UTC
    target_tz = pytz.timezone("Africa/Nairobi")
    # Iterate through and update the 'Played_At' list in the dictionary
    for i, timestamp in enumerate(song_dict["Played_At"]):
        utc_datetime = datetime.datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%fZ")
        utc_datetime = source_tz.localize(utc_datetime)

        # Convert to the target time zone (EAT)
        eat_datetime = utc_datetime.astimezone(target_tz)

        # Format the datetime in the desired format
        formatted_time = eat_datetime.strftime("%H:%M:%S")
        song_dict["Played_At"][i] = formatted_time

    # Check if any of the timestamps are within the last 24 hours
    df = pd.DataFrame(song_dict, columns = ['Track', 'Artist', 'Played_At', 'Date'])
    
    if check_valid_data(df):
        print("Data valid, Proceed to Load Stage!")
        
        
    DB_LOCATION = "sqlite:///path/database.db"
    engine = sqlalchemy.create_engine(DB_LOCATION)
    conn = sqlite3.connect("spotify_db.db")
    cursor = conn.cursor()
    
    # cursor.execute("DROP TABLE spotify.spotify_tb")
    
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS spotify_tb (
    Track text,
    Artist text,
    Played_At TIME ,
    Date DATE,
    CONSTRAINT primary_key_constraint PRIMARY KEY (Played_At)
    );""")
    print("Database Open Successfully")
    
    try:
        df.to_sql("spotify_tb", engine ,index='False', if_exists='append')
    except:
        print("Data already exists in the database")
        
    conn.close()
    print("Database closed successfully")
      

In [ ]:
json_result

In [ ]:
df